In [63]:
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import os
from glob import glob
from GANs import Discriminator, Generator
from tqdm import tqdm
os.getcwd(), os.listdir()

('/home/pyf/ML/GAN/Painting GAN',
 ['.ipynb_checkpoints',
  'GANs.py',
  'Painting',
  'Untitled.ipynb',
  '__pycache__'])

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [66]:
# Declaring GLOBAL VARS:
BATCH_SIZE = 5
IMG_SHAPE = 208
CHANNELS = 3
EPOCHS = 5
FEATURES = 16
RANDOM_NOISE_SHAPE = 256

In [42]:
class PaintingDataset(Dataset):
    def __init__(self, loc = os.path.join('Painting', 'train_2'), img_shape = 208):
        self.paintings = glob(os.path.join(loc, '')+'*.jpg')
        self.img_shape = img_shape
        
    def __len__(self):
        return len(self.paintings)
    
    def __getitem__(self, idx):
        img = self.paintings[idx]
        img = cv2.imread(img)
        img = cv2.resize(img, (self.img_shape, self.img_shape))
        img = np.moveaxis(img, -1, 0) # pytorch takes channel first images
        img = torch.tensor(img).float()
        return img.to(device)
        
ds = PaintingDataset(img_shape = IMG_SHAPE)
ds[1].shape # -> torch.Size([3, 208, 208])
loader = DataLoader(ds, shuffle = True, batch_size = BATCH_SIZE)
batch = next(iter(loader))
batch.shape # -> torch.Size([5,3, 208, 208])

torch.Size([5, 3, 208, 208])

In [67]:
disc = Discriminator().to(device)
gen = Generator().to(device)

lr = 0.0002
epochs = 2

optimD = optim.Adam(disc.parameters(), lr = lr, betas = (0.5, 0.99))
optimG = optim.Adam(gen.parameters(), lr = lr, betas = (0.5, 0.99))

criterion = nn.BCELoss()

real_label = 1
fake_label = 0

# fixed_noise = torch.randn(batch_size, channels_noise, 1, 1).to(device)

for epoch in tqdm(range(epochs)):
    for data in loader:
        batch_size = data.shape[0]

        ## Training discriminator

        '''First feeding the real images'''
        label = (torch.ones(batch_size)* 0.9).to(device)
        output = disc(data).reshape(-1)
        lossD_real = criterion(output, label)

        '''feeding generated images'''
        label = (torch.ones(batch_size) * 0.1).to(device)
        rand_noise = torch.randn((batch_size, RANDOM_NOISE_SHAPE, 1, 1)).to(device)
        fake_image = gen(rand_noise)
        output = disc(fake_image.detach()).reshape(-1)
        lossD_fake = criterion(output, label)

        '''Back propogating discriminator and updating weights'''
        disc.zero_grad()
        lossD = lossD_real + lossD_fake
        lossD.backward()
        optimD.step()

        ## Training generator

        label = torch.ones(batch_size).to(device)
        output = disc(fake_image).reshape(-1)
        lossG = criterion(output, label)

        '''Backpropogating'''
        gen.zero_grad()
        lossG.backward()
        optimG.step()

  0%|          | 0/2 [07:32<?, ?it/s]


KeyboardInterrupt: 